# Variant Calling

Now that we have the aligned sequences to the reference, we can look for difference between the aligned sequences and the reference. This process is also known as variant calling.

Let's review the files that we have.

In [1]:
ls -lh

total 482M
-rw-r--r-- 1 root root  15K Jul 23 14:48 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root  588 Jul 23 14:48  chr5.fa.amb
-rw-r--r-- 1 root root   44 Jul 23 14:48  chr5.fa.ann
-rw-r--r-- 1 root root 174M Jul 23 14:48  chr5.fa.bwt
-rw-r--r-- 1 root root  44M Jul 23 14:48  chr5.fa.pac
-rw-r--r-- 1 root root  87M Jul 23 14:49  chr5.fa.sa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip
-rw-r--r-- 1 root root 963K Jul 23 14:49  mapped.sam


## Preparing for variant calling

In order to perform variant calling, we need to preprocess the SAM file as well as the reference chromosome 5 fasta file. The following steps will be performed :

* Converting SAM to BAM format (this is the binary compressed version)
* Sorting the BAM file
* Indexing the BAM file
* Indexing the reference fasta file


### Processing the SAM file generated by BWA

We will process the SAM file using samtools, which can be accessed after loading the module.

In [2]:
samtools view


Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]

Options:
  -b       output BAM
  -C       output CRAM (requires -T)
  -1       use fast BAM compression (implies -b)
  -u       uncompressed BAM output (implies -b)
  -h       include header in SAM output
  -H       print SAM header only (no alignments)
  -c       print only the count of matching records
  -o FILE  output file name [stdout]
  -U FILE  output reads not selected by filters to FILE [null]
  -t FILE  FILE listing reference names and lengths (see long help) [null]
  -L FILE  only include reads overlapping this BED FILE [null]
  -r STR   only include reads in read group STR [null]
  -R FILE  only include reads with read group listed in FILE [null]
  -q INT   only include reads with mapping quality >= INT [0]
  -l STR   only include reads in library STR [null]
  -m INT   only include reads with number of CIGAR operations consuming
           query sequence >= INT [0]
  -f INT   only include reads with a

We will begin by converting the SAM file to the compressed BAM format. To do this, we will need to refer to the reference fasta file and the SAM file. The output will be redirected to a new BAM file.

In [3]:
samtools view -bT chr5.fa mapped.sam > mapped.bam

[samfaipath] build FASTA index...


In [4]:
ls -lh

total 483M
-rw-r--r-- 1 root root  15K Jul 23 14:48 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root  588 Jul 23 14:48  chr5.fa.amb
-rw-r--r-- 1 root root   44 Jul 23 14:48  chr5.fa.ann
-rw-r--r-- 1 root root 174M Jul 23 14:48  chr5.fa.bwt
-rw-r--r-- 1 root root   23 Jul 23 14:49  chr5.fa.fai
-rw-r--r-- 1 root root  44M Jul 23 14:48  chr5.fa.pac
-rw-r--r-- 1 root root  87M Jul 23 14:49  chr5.fa.sa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip
-rw-r--r-- 1 root root 212K Jul 23 14:49  mapped.bam
-rw-r--r-- 1 root root 963K Jul 23 14:49  mapped.sam


Notice that the BAM files is much smaller than the SAM file

Next, we will sort and index the BAM file

In [5]:
samtools sort -o mapped.sort.bam mapped.bam 
samtools index mapped.sort.bam

In [6]:
ls -lh

total 483M
-rw-r--r-- 1 root root  15K Jul 23 14:48 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root  588 Jul 23 14:48  chr5.fa.amb
-rw-r--r-- 1 root root   44 Jul 23 14:48  chr5.fa.ann
-rw-r--r-- 1 root root 174M Jul 23 14:48  chr5.fa.bwt
-rw-r--r-- 1 root root   23 Jul 23 14:49  chr5.fa.fai
-rw-r--r-- 1 root root  44M Jul 23 14:48  chr5.fa.pac
-rw-r--r-- 1 root root  87M Jul 23 14:49  chr5.fa.sa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip
-rw-r--r-- 1 root root 212K Jul 23 14:49  mapped.bam
-rw-r--r-- 1 root root 963K Jul 23 14:49  mapped.sam
-rw-r--r-- 1 root

### Indexing the reference fasta file

To prepare the reference fasta file for variant calling, we need to index the file

In [7]:
samtools faidx chr5.fa

In [8]:
ls -lh

total 483M
-rw-r--r-- 1 root root  15K Jul 23 14:48 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root  588 Jul 23 14:48  chr5.fa.amb
-rw-r--r-- 1 root root   44 Jul 23 14:48  chr5.fa.ann
-rw-r--r-- 1 root root 174M Jul 23 14:48  chr5.fa.bwt
-rw-r--r-- 1 root root   23 Jul 23 14:50  chr5.fa.fai
-rw-r--r-- 1 root root  44M Jul 23 14:48  chr5.fa.pac
-rw-r--r-- 1 root root  87M Jul 23 14:49  chr5.fa.sa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip
-rw-r--r-- 1 root root 212K Jul 23 14:49  mapped.bam
-rw-r--r-- 1 root root 963K Jul 23 14:49  mapped.sam
-rw-r--r-- 1 root

Notice the .fai file

In [9]:
head chr5.fa.fai

chr5	181538259	6	50	51


## Variant calling using Platypus

There are different methods for variant calling. Here, we will use Platypus, a haplotype-based variant caller that increases the sensitivity and specificity of the varianta calls (http://www.well.ox.ac.uk/platypus)

<img src="https://bchdb.nus.edu.sg/media/notebook/ng.3036-F1.jpg" width=800></img>

In [10]:
freebayes -h

usage: freebayes [OPTION] ... [BAM FILE] ... 

Bayesian haplotype-based polymorphism discovery.

citation: Erik Garrison, Gabor Marth
          "Haplotype-based variant detection from short-read sequencing"
          arXiv:1207.3907 (http://arxiv.org/abs/1207.3907)

overview:

    To call variants from aligned short-read sequencing data, supply BAM files and
    a reference.  FreeBayes will provide VCF output on standard out describing SNPs,
    indels, and complex variants in samples in the input alignments.

    By default, FreeBayes will consider variants supported by at least 2
    observations in a single sample (-C) and also by at least 20% of the reads from
    a single sample (-F).  These settings are suitable to low to high depth
    sequencing in haploid and diploid samples, but users working with polyploid or
    pooled samples may wish to adjust them depending on the characteristics of
    their sequencing data.

    FreeBayes is capable of calling variant haplotypes shorte

The variant caller requires the reference file and the BAM file with its index.

In [11]:
freebayes -f chr5.fa mapped.sort.bam > result.vcf

Let's look at the first and last 50 lines of the VCF file

In [12]:
head -n 50 result.vcf
tail -n 50 result.vcf

##fileformat=VCFv4.2
##fileDate=20200723
##source=freeBayes v1.3.2-dirty
##reference=chr5.fa
##contig=<ID=chr5,length=181538259>
##phasing=none
##commandline="freebayes -f chr5.fa mapped.sort.bam"
##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of samples with data">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total read depth at the locus">
##INFO=<ID=DPB,Number=1,Type=Float,Description="Total read depth per bp at the locus; bases in reads overlapping / bases in haplotype">
##INFO=<ID=AC,Number=A,Type=Integer,Description="Total number of alternate alleles in called genotypes">
##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles in called genotypes">
##INFO=<ID=AF,Number=A,Type=Float,Description="Estimated allele frequency in the range (0,1]">
##INFO=<ID=RO,Number=1,Type=Integer,Description="Count of full observations of the reference haplotype.">
##INFO=<ID=AO,Number=A,Type=Integer,Description="Count of full observations of this alternate haplotype.">